<a href="https://colab.research.google.com/github/gaboaurora/Low-Resource-Translation/blob/main/ingles_kiche.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Traducción Inglés -> Kiche


## Configuración inicial

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# ===================================================================
# CELDA DE CONFIGURACIÓN CENTRAL DEL PROYECTO
# ===================================================================
# Para cambiar de experimento, solo necesitan modificar esta celda.

import os

# --- 1. Definición de la Tarea de Traducción ---
# Idioma de origen (el que no es español)
SOURCE_LANG_NAME = "Inglés"
SOURCE_LANG_CODE = "eng_Latn"

# Idioma de destino (la lengua maya que están estudiando)
TARGET_LANG_NAME = "K'iche'"
TARGET_LANG_CODE = "quc_Latn" # Código correcto para K'iche'
TARGET_LANG_SHORT_CODE = "quc" # Código del dataset MayanV

# Idioma pivote (el que se usa como puente)
PIVOT_LANG_NAME = "Español"
PIVOT_LANG_CODE = "spa_Latn"
PIVOT_LANG_SHORT_CODE = "es" # Código del dataset MayanV

# --- 2. Parámetros del Modelo y Entrenamiento ---
BASE_MODEL_ID = "facebook/nllb-200-distilled-600M"

# --- 3. Rutas para Guardar en Google Drive ---
# Asegúrate de que esta carpeta exista en tu Google Drive
DRIVE_PATH = "/content/drive/MyDrive/Colab Notebooks/Proyecto_DL_Traduccion"
ES_QUC_DATASET_PATH = f"{DRIVE_PATH}/es_quc_dataset"
MAYANV_REPO_PATH = f"{DRIVE_PATH}/mayanv"
EN_QUC_DATASET_FILES_PATH = f"{DRIVE_PATH}/en_quc_dataset_files"
os.makedirs(DRIVE_PATH, exist_ok=True) # Crea la carpeta si no existe

# Nombres de los modelos que se guardarán
FINETUNED_PIVOT_MODEL_PATH = f"{DRIVE_PATH}/nllb_finetuned_{PIVOT_LANG_SHORT_CODE}_{TARGET_LANG_SHORT_CODE}"
FINETUNED_SYNTHETIC_MODEL_PATH = f"{DRIVE_PATH}/nllb_finetuned_{SOURCE_LANG_NAME.lower()}_{TARGET_LANG_SHORT_CODE}"

print(f"✅ Configuración cargada para la tarea: {SOURCE_LANG_NAME} -> {TARGET_LANG_NAME}")
print(f"Los modelos se guardarán en: {DRIVE_PATH}")

✅ Configuración cargada para la tarea: Inglés -> K'iche'
Los modelos se guardarán en: /content/drive/MyDrive/Colab Notebooks/Proyecto_DL_Traduccion


### Funciones comunes

In [3]:
def translate_text(model, tokenizer, text, src_lang, tgt_lang):
    tokenizer.src_lang = src_lang
    forced_bos_id = tokenizer.convert_tokens_to_ids(tgt_lang)
    inputs = tokenizer(text, return_tensors="pt", truncation=True)
    outputs = model.generate(**inputs, forced_bos_token_id=forced_bos_id)
    return tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

In [4]:
def generate_dataset(language, train_folder="train", test_folder="test", base_path=MAYANV_REPO_PATH): # Use MAYANV_REPO_PATH from config
    """
    Generates a dataset from text files within the mayanv repository.

    Args:
        language (str): The language code (e.g., "quc").
        train_folder (str): The name of the training data folder within the language directory.
        test_folder (str): The name of the testing data folder within the language directory.
        base_path (str): The base path to the cloned mayanv repository in Google Drive.

    Returns:
        DatasetDict: A DatasetDict containing the train and test splits.
    """
    # Construct paths to the text files within the mayanv repository in Drive
    mayanv_language_path = os.path.join(base_path, "MayanV", language)
    train_lang_path = os.path.join(mayanv_language_path, train_folder, f"data.{language}")
    test_lang_path = os.path.join(mayanv_language_path, test_folder, f"data.{language}")

    train_es_path = os.path.join(mayanv_language_path, train_folder, "data.es")
    test_es_path = os.path.join(mayanv_language_path, test_folder, "data.es")

    print(f"Attempting to load train split from: {train_es_path} and {train_lang_path}")
    print(f"Attempting to load test split from: {test_es_path} and {test_lang_path}")


    # Carga manual de archivos
    def load_lines(path):
        if not os.path.exists(path):
            print(f"❌ Error: File not found at {path}")
            return None
        with open(path, encoding="utf-8") as f:
            return [line.strip() for line in f if line.strip()] # Filter out empty lines


    train_src = load_lines(train_lang_path)
    train_tgt = load_lines(train_es_path)

    test_src = load_lines(test_lang_path)
    test_tgt = load_lines(test_es_path)

    # Check if loading was successful
    if train_src is None or train_tgt is None or test_src is None or test_tgt is None:
        return None

    # Crea datasets
    train_dataset = Dataset.from_dict({"input": train_src, "target": train_tgt})
    test_dataset = Dataset.from_dict({"input": test_src, "target": test_tgt})

    return DatasetDict({
        "train": train_dataset,
        "test": test_dataset,
    })

## Librerías

In [5]:
from IPython.display import clear_output
!pip install -U bitsandbytes accelerate peft transformers sentencepiece datasets
!pip install evaluate
!pip install rouge_score
clear_output()

In [6]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Trainer, TrainingArguments, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, TaskType
from datasets import load_dataset, DatasetDict, Dataset
import evaluate
from tqdm import tqdm
import csv
import torch

In [7]:
from peft import PeftModel

In [8]:
!git clone https://github.com/transducens/mayanv.git

Cloning into 'mayanv'...
remote: Enumerating objects: 177, done.
remote: Counting objects: 100% (177/177), done.
remote: Compressing objects: 100% (163/163), done.
remote: Total 177 (delta 22), reused 157 (delta 14), pack-reused 0 (from 0)
Receiving objects: 100% (177/177), 1.35 MiB | 3.88 MiB/s, done.
Resolving deltas: 100% (22/22), done.


## Datasets

### Verificacion inicial de lenguaje en el modelo NLLB

In [ ]:
from transformers import AutoTokenizer

# Cargamos el tokenizador oficial de NLLB
model_name = "facebook/nllb-200-distilled-600M"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# --- Verifiquemos los idiomas ---

kiche_code = "quc_Latn"
sipakapense_code = "qum_Latn"

print(f"Buscando el ID para K'iche' ({kiche_code})...")
try:
    kiche_id = tokenizer.convert_tokens_to_ids(kiche_code)
    print(f"✅ ÉXITO: El ID para '{kiche_code}' es: {kiche_id}. ¡SÍ EXISTE!")
except Exception as e:
    print(f"❌ FALLO: No se pudo encontrar un ID para '{kiche_code}'. ¡NO EXISTE!")

print("-" * 20)

print(f"Buscando el ID para Sipakapense ({sipakapense_code})...")
try:
    # El tokenizador arroja un error si no encuentra el token.
    # Usamos .get para manejarlo sin que el programa se caiga.
    sipakapense_id = tokenizer.vocab.get(sipakapense_code)
    if sipakapense_id is not None:
         print(f"✅ ÉXITO: El ID para '{sipakapense_code}' es: {sipakapense_id}. ¡SÍ EXISTE!")
    else:
         print(f"❌ FALLO: No se encontró un ID para '{sipakapense_code}'. ¡NO EXISTE!")
except Exception as e:
    print(f"❌ FALLO: Ocurrió un error buscando '{sipakapense_code}'.")

Buscando el ID para K'iche' (quc_Latn)...
✅ ÉXITO: El ID para 'quc_Latn' es: 3. ¡SÍ EXISTE!
--------------------
Buscando el ID para Sipakapense (qum_Latn)...
❌ FALLO: No se encontró un ID para 'qum_Latn'. ¡NO EXISTE!


### Siguientes pasos

Dataset se puede encontrar en el [siguiente enlace](https://github.com/transducens/mayanv).

In [ ]:
es_quc_dataset = generate_dataset("quc", train_folder="train", test_folder="test")
es_quc_dataset

Attempting to load train split from: /content/drive/MyDrive/Colab Notebooks/Proyecto_DL_Traduccion/mayanv/MayanV/quc/train/data.es and /content/drive/MyDrive/Colab Notebooks/Proyecto_DL_Traduccion/mayanv/MayanV/quc/train/data.quc
Attempting to load test split from: /content/drive/MyDrive/Colab Notebooks/Proyecto_DL_Traduccion/mayanv/MayanV/quc/test/data.es and /content/drive/MyDrive/Colab Notebooks/Proyecto_DL_Traduccion/mayanv/MayanV/quc/test/data.quc


DatasetDict({
    train: Dataset({
        features: ['input', 'target'],
        num_rows: 624
    })
    test: Dataset({
        features: ['input', 'target'],
        num_rows: 1000
    })
})

In [ ]:
# Save the dataset to the drive path
es_quc_dataset.save_to_disk(f"{DRIVE_PATH}/es_quc_dataset")

# Save the cloned git repository to the drive path
!cp -r mayanv/ "{DRIVE_PATH}/mayanv"


Saving the dataset (0/1 shards):   0%|          | 0/624 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1000 [00:00<?, ? examples/s]

^C


## Preparación y Entrenamiento de Modelos


### 1. Modelo Zero-shot

In [9]:
model_name = "facebook/nllb-200-distilled-600M"
zeroshot_tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
zeroshot_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [10]:
zeroshot_tokenizer.src_lang = "eng_Latn"
forced_bos_id = zeroshot_tokenizer.convert_tokens_to_ids("quc_Latn")

### 2. Modelo con Finetuning Español-Kiche

Aquí entrenamos un modelo para que sea un experto en una tarea específica: Español -> K'iche'. Este modelo se usará después como una pieza en el experimento del pipeline.

#### 2.1 Modelo con Finetuning Español-Kiche

In [ ]:
model_name = "facebook/nllb-200-distilled-600M"
finetune_tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)


bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)


finetune_model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)

# Configurar LoRA
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM
)

# Aplicar QLoRA
finetune_model = get_peft_model(finetune_model, lora_config)

In [ ]:
def preprocess_function(examples):
    max_length = 128
    # Tokeniza el input
    inputs = finetune_tokenizer(examples["input"], max_length=max_length, truncation=True, padding=True)
    # Tokeniza la salida (target)
    targets = finetune_tokenizer(examples["target"], max_length=max_length, truncation=True, padding=True)

    inputs["labels"] = targets["input_ids"]
    return inputs

tokenized_train = es_quc_dataset["train"].map(preprocess_function, batched=True)
tokenized_test = es_quc_dataset["test"].map(preprocess_function, batched=True)

Map:   0%|          | 0/624 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
training_args = TrainingArguments(
    output_dir="./qlora_finetuned_qum_nllb",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    learning_rate=2e-4,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs_qlora",
    save_total_limit=2,
    bf16=False,
    fp16=True,
    report_to="none"
)

trainer = Trainer(
    model=finetune_model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=finetune_tokenizer,
)

trainer.train()

# Define the save path
save_path = f"{DRIVE_PATH}/qlora_finetuned_es_quc_nllb"

# Save the model and tokenizer to the specified path
finetune_model.save_pretrained(save_path)
finetune_tokenizer.save_pretrained(save_path)

print(f"✅ Model and tokenizer saved to {save_path}")

/tmp/ipython-input-16-279613285.py:16: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForSeq2SeqLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Epoch,Training Loss,Validation Loss
1,No log,5.500633
2,No log,5.436388
3,No log,5.419795


✅ Model and tokenizer saved to /content/drive/MyDrive/Colab Notebooks/Proyecto_DL_Traduccion/qlora_finetuned_es_quc_nllb


### 3. Modelo con Finetuning Inglés-Kiche

#### 3.1 CREACIÓN DEL DATASET SINTÉTICO usando el modelo Zero-Shot

Tenemos que fábricar las palabras nuevas, convertir un par (Español, K'iche') en un par (Inglés, K'iche'). Para esto usamos el modelo Zero-Shot para obtener las traducciones pertinentes

In [14]:
def build_en_quc_split(split, model, tokenizer, dataset):

    print(dataset["train"][8])
    new_inputs = []
    new_targets = []

    print(f"🔄 Traduciendo split '{split}' de español a inglés...\n")
    for example in tqdm(dataset[split]):
        es_text = example["target"]
        en_text = translate_text(model, tokenizer, es_text, "spa_Latn", "eng_Latn")
        new_inputs.append(en_text)
        new_targets.append(example["target"])

    return Dataset.from_dict({
        "input": new_inputs,
        "target": new_targets
    })

# Generar splits
dataset_variable_name = f"es_{TARGET_LANG_SHORT_CODE}_dataset"
es_target_dataset = globals().get(dataset_variable_name)

if es_target_dataset is None:
    raise NameError(f"Dataset variable '{dataset_variable_name}' not found. Please ensure the dataset for {PIVOT_LANG_NAME}-{TARGET_LANG_NAME} is generated and named correctly.")


en_quc_train = build_en_quc_split("train", zeroshot_model, zeroshot_tokenizer, es_target_dataset)
en_quc_test = build_en_quc_split("test", zeroshot_model, zeroshot_tokenizer, es_target_dataset)

# Combinar
en_quc_dataset = DatasetDict({
    "train": en_quc_train,
    "test": en_quc_test
})

{'input': 'Ri tat Xwan potzʼ', 'target': '#quc# Don Juan está ciego'}
🔄 Traduciendo split 'train' de español a inglés...



100%|██████████| 624/624 [25:07<00:00,  2.42s/it]


{'input': 'Ri tat Xwan potzʼ', 'target': '#quc# Don Juan está ciego'}
🔄 Traduciendo split 'test' de español a inglés...



100%|██████████| 1000/1000 [39:07<00:00,  2.35s/it]


#### 3.2 SAVE Dataset

In [15]:
def save_en_quc_dataset_to_files(dataset: DatasetDict, output_directory: str):
    """
    Saves the English-K'iche' dataset splits (train, test) into separate .en and .quc files.

    Args:
        dataset (DatasetDict): The dataset object containing 'train' and 'test' splits,
                               where each example has 'input' (English) and 'target' (K'iche').
        output_directory (str): The path to the directory where the files will be saved.
                                This directory will be created if it doesn't exist.
    """
    if not isinstance(dataset, DatasetDict):
        print(f"❌ Error: Expected 'dataset' to be a DatasetDict, but got {type(dataset)}")
        return

    # Create the output directory if it doesn't exist
    os.makedirs(output_directory, exist_ok=True)
    print(f"Ensuring output directory exists: {output_directory}")

    try:
        # Save each split (train, test) into .en (English) and .quc (K'iche') files
        for split in ["train", "test"]:
            if split not in dataset:
                print(f"⚠️ Warning: Split '{split}' not found in the dataset. Skipping.")
                continue

            # Check if 'input' and 'target' keys exist in the first example of the split
            # This is a basic check to ensure the expected structure
            if dataset[split] and ("input" not in dataset[split][0] or "target" not in dataset[split][0]):
                print(f"❌ Error: Expected 'input' and 'target' keys in dataset[{split}] examples. Skipping.")
                continue

            file_path_en = os.path.join(output_directory, f"{split}.en")
            file_path_quc = os.path.join(output_directory, f"{split}.quc")

            with open(file_path_en, "w", encoding="utf-8") as f_en, \
                 open(file_path_quc, "w", encoding="utf-8") as f_quc:

                for example in dataset[split]:
                    f_en.write(example["input"].strip() + "\n")
                    f_quc.write(example["target"].strip() + "\n")

            print(f"✅ Files '{split}.en' and '{split}.quc' saved to {output_directory}")

        print(f"✅ All .en and .quc files processed and saved to {output_directory}")

    except Exception as e:
        print(f"❌ An error occurred while saving the files: {e}")



In [16]:
save_en_quc_dataset_to_files(en_quc_dataset, EN_QUC_DATASET_FILES_PATH)

Ensuring output directory exists: /content/drive/MyDrive/Colab Notebooks/Proyecto_DL_Traduccion/en_quc_dataset_files
✅ Files 'train.en' and 'train.quc' saved to /content/drive/MyDrive/Colab Notebooks/Proyecto_DL_Traduccion/en_quc_dataset_files
✅ Files 'test.en' and 'test.quc' saved to /content/drive/MyDrive/Colab Notebooks/Proyecto_DL_Traduccion/en_quc_dataset_files
✅ All .en and .quc files processed and saved to /content/drive/MyDrive/Colab Notebooks/Proyecto_DL_Traduccion/en_quc_dataset_files


#### 3.3 Preparación y entrenamiento

In [ ]:
model_name = "facebook/nllb-200-distilled-600M"
en_quc_tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
en_quc_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [ ]:
def preprocess_function_en_quc(examples):
    max_length = 128
    inputs = en_quc_tokenizer(examples["input"], max_length=max_length, truncation=True, padding="max_length")
    targets = en_quc_tokenizer(examples["target"], max_length=max_length, truncation=True, padding="max_length")
    inputs["labels"] = targets["input_ids"]
    return inputs

tokenized_en_quc_train = en_quc_dataset["train"].map(preprocess_function_en_quc, batched=True)
tokenized_en_quc_test = en_quc_dataset["test"].map(preprocess_function_en_quc, batched=True)

Map:   0%|          | 0/624 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
training_args_en_quc = TrainingArguments(
    output_dir=f"{DRIVE_PATH}/finetuned_en_quc_nllb",
    eval_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    save_strategy="epoch",
    logging_dir="./logs_en_quc",
    logging_steps=100,
    report_to="none" # Disable reporting to wandb
)

"""
training_args = TrainingArguments(
    output_dir="./qlora_finetuned_qum_nllb",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    learning_rate=2e-4,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs_qlora",
    save_total_limit=2,
    bf16=False,
    fp16=True,
    report_to="none"
)
"""

'\ntraining_args = TrainingArguments(\n    output_dir="./qlora_finetuned_qum_nllb",\n    per_device_train_batch_size=4,\n    per_device_eval_batch_size=4,\n    num_train_epochs=3,\n    learning_rate=2e-4,\n    eval_strategy="epoch",\n    save_strategy="epoch",\n    logging_dir="./logs_qlora",\n    save_total_limit=2,\n    bf16=False,\n    fp16=True,\n    report_to="none"\n)\n'

In [ ]:
"""


trainer = Trainer(
    model=finetune_model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=finetune_tokenizer,
)
"""


trainer_en_quc = Trainer(
    model=en_quc_model,
    args=training_args_en_quc,
    train_dataset=tokenized_en_quc_train,
    eval_dataset=tokenized_en_quc_test,
    tokenizer=en_quc_tokenizer,
)



trainer_en_quc.train()

# Define the save path for the EN-QUC model
save_path_en_quc = f"{DRIVE_PATH}/finetuned_en_quc_nllb"

# Save the model and tokenizer to the specified path
en_quc_model.save_pretrained(save_path_en_quc)
en_quc_tokenizer.save_pretrained(save_path_en_quc)

print(f"✅ EN-QUC model and tokenizer saved to {save_path_en_quc}")

/tmp/ipython-input-56-489488295.py:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_en_quc = Trainer(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.58.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,No log,6.088306
2,8.219700,4.663156
3,5.058900,4.226790


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3685: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 200}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


✅ EN-QUC model and tokenizer saved to /content/drive/MyDrive/Colab Notebooks/Proyecto_DL_Traduccion/finetuned_en_quc_nllb


## Load from Google Drive

#### Load English Kiche Zero-Shot Model

In [20]:
model_name = "facebook/nllb-200-distilled-600M"
zeroshot_tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
zeroshot_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

zeroshot_tokenizer.src_lang = "eng_Latn"
forced_bos_id = zeroshot_tokenizer.convert_tokens_to_ids("quc_Latn")

#### Load Spanish Kiche Model

In [8]:
def load_models_and_tokenizers_espanol_kiche(drive_path: str, base_model_id: str):
    """
    Loads the fine-tuned Español-K'iche' QLoRA model and tokenizer.

    Args:
        drive_path (str): Base path to your Google Drive.
        base_model_id (str): Identifier for the base model (e.g., "facebook/nllb-200-distilled-600M").

    Returns:
        tuple: A tuple containing reloaded_finetune_model and reloaded_finetune_tokenizer.
               Returns (None, None) if loading fails.
    """
    reloaded_finetune_model = None
    reloaded_finetune_tokenizer = None
    save_path_es_quc = f"{drive_path}/qlora_finetuned_es_quc_nllb"

    if os.path.exists(save_path_es_quc):
        print(f"✅ Found fine-tuned model directory at {save_path_es_quc}")
        try:
            # Load the base model first (needed for QLoRA)
            reloaded_base_model = AutoModelForSeq2SeqLM.from_pretrained(base_model_id)

            # Load the QLoRA adapter weights on top of the base model
            reloaded_finetune_model = PeftModel.from_pretrained(reloaded_base_model, save_path_es_quc)

            # Load the tokenizer
            reloaded_finetune_tokenizer = AutoTokenizer.from_pretrained(save_path_es_quc)

            print(f"✅ Model and tokenizer reloaded successfully from {save_path_es_quc}")
        except Exception as e:
            print(f"❌ Error reloading fine-tuned model: {e}")
    else:
        print(f"❌ Fine-tuned model directory not found at {save_path_es_quc}")
        print("Please ensure the model was saved correctly.")

    return reloaded_finetune_model, reloaded_finetune_tokenizer

In [9]:
# Load models and tokenizers
espanol_kiche_finetune_model, espanol_kiche_finetune_tokenizer = load_models_and_tokenizers_espanol_kiche(DRIVE_PATH, BASE_MODEL_ID)
es_quc_model = espanol_kiche_finetune_model
es_quc_tokenizer = espanol_kiche_finetune_tokenizer

✅ Found fine-tuned model directory at /content/drive/MyDrive/Colab Notebooks/Proyecto_DL_Traduccion/qlora_finetuned_es_quc_nllb
✅ Model and tokenizer reloaded successfully from /content/drive/MyDrive/Colab Notebooks/Proyecto_DL_Traduccion/qlora_finetuned_es_quc_nllb


#### Load English Kiche Model

In [10]:
from peft import PeftModel
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM # Import AutoModelForSeq2SeqLM

def load_finetuned_english_kiche_model(drive_path: str, base_model_id: str):
    """
    Loads the fine-tuned English-K'iche' model and tokenizer from Google Drive.

    Args:
        drive_path (str): Base path to your Google Drive where the model is saved.
        base_model_id (str): Identifier for the base model (e.g., "facebook/nllb-200-distilled-600M").

    Returns:
        tuple: A tuple containing reloaded_model and reloaded_tokenizer.
               Returns (None, None) if loading fails.
    """
    reloaded_model = None
    reloaded_tokenizer = None
    save_path = f"{drive_path}/finetuned_en_quc_nllb"

    if os.path.exists(save_path):
        print(f"✅ Found fine-tuned model directory at {save_path}")
        try:
            # Load the fine-tuned model directly
            reloaded_model = AutoModelForSeq2SeqLM.from_pretrained(save_path)

            # Load the tokenizer
            reloaded_tokenizer = AutoTokenizer.from_pretrained(save_path)

            print(f"✅ Model and tokenizer reloaded successfully from {save_path}")
        except Exception as e:
            print(f"❌ Error reloading fine-tuned model: {e}")
            reloaded_model = None
            reloaded_tokenizer = None
    else:
        print(f"❌ Fine-tuned model directory not found at {save_path}")
        print("Please ensure the model was saved correctly.")

    return reloaded_model, reloaded_tokenizer

# Example usage (you can run this cell to test loading)
# en_quc_finetuned_model, en_quc_finetuned_tokenizer = load_finetuned_english_kiche_model(DRIVE_PATH, BASE_MODEL_ID)

In [11]:
english_kiche_finetuned_model, english_kiche_finetuned_tokenizer = load_finetuned_english_kiche_model(DRIVE_PATH, BASE_MODEL_ID)
en_quc_model = english_kiche_finetuned_model
en_quc_tokenizer = english_kiche_finetuned_tokenizer

✅ Found fine-tuned model directory at /content/drive/MyDrive/Colab Notebooks/Proyecto_DL_Traduccion/finetuned_en_quc_nllb
✅ Model and tokenizer reloaded successfully from /content/drive/MyDrive/Colab Notebooks/Proyecto_DL_Traduccion/finetuned_en_quc_nllb


#### Load Spanish-Kiche and English-Kiche datasets

In [12]:
def load_datasets(es_quc_dataset_path: str, en_quc_dataset_files_path: str, mayanv_repo_path: str):
    """
    Carga ambos datasets desde sus respectivas carpetas guardadas en Google Drive.
    """
    loaded_es_quc_dataset = None
    en_quc_dataset = None

    # --- 1. Cargar el dataset Español-K'iche' desde su carpeta guardada ---
    # Se mantiene tu lógica original de cargar desde el disco.
    print(f"\n--- Intentando cargar es_quc_dataset desde: {es_quc_dataset_path} ---")
    try:
        loaded_es_quc_dataset = DatasetDict.load_from_disk(es_quc_dataset_path)
        print("✅ Dataset Español-K'iche' cargado exitosamente desde el disco.")
        print(loaded_es_quc_dataset)
    except Exception as e:
        print(f"❌ Error al cargar es_quc_dataset desde el disco: {e}")
        print("Asegúrate de haber ejecutado la celda que lo guarda con 'save_to_disk' primero.")

    # --- 2. Cargar el dataset sintético Inglés-K'iche' desde archivos de texto ---
    # Aquí aplicamos la corrección para que se cargue como un dataset paralelo.
    print(f"\n--- Cargando dataset sintético desde: {en_quc_dataset_files_path} ---")

    def load_parallel_corpus_from_files(split_name):
        """Función interna para leer los archivos .en y .quc y unirlos."""
        source_path = f"{en_quc_dataset_files_path}/{split_name}.en"
        target_path = f"{en_quc_dataset_files_path}/{split_name}.quc"

        if not os.path.exists(source_path) or not os.path.exists(target_path):
            print(f"ADVERTENCIA: Archivos para el split '{split_name}' no encontrados en la ruta.")
            return None

        with open(source_path, "r", encoding="utf-8") as f:
            source_lines = [line.strip() for line in f.readlines()]
        with open(target_path, "r", encoding="utf-8") as f:
            target_lines = [line.strip() for line in f.readlines()]

        if len(source_lines) != len(target_lines):
            raise ValueError(f"Archivos del split '{split_name}' no tienen el mismo número de líneas.")

        # Devolvemos un Dataset con las columnas "input" y "target"
        return Dataset.from_dict({"input": source_lines, "target": target_lines})

    train_split = load_parallel_corpus_from_files("train")
    test_split = load_parallel_corpus_from_files("test")

    if train_split and test_split:
        en_quc_dataset = DatasetDict({"train": train_split, "test": test_split})
        print("✅ Dataset sintético Inglés-K'iche' cargado correctamente con 2 columnas.")
    else:
        print("❌ Fallo al cargar el dataset sintético.")
        en_quc_dataset = None

    return loaded_es_quc_dataset, en_quc_dataset

In [13]:
# Load datasets and verify repository
espanol_kiche_dataset, english_kiche_dataset = load_datasets(ES_QUC_DATASET_PATH, EN_QUC_DATASET_FILES_PATH, MAYANV_REPO_PATH)
es_quc_dataset = espanol_kiche_dataset
en_quc_dataset = english_kiche_dataset


--- Intentando cargar es_quc_dataset desde: /content/drive/MyDrive/Colab Notebooks/Proyecto_DL_Traduccion/es_quc_dataset ---
✅ Dataset Español-K'iche' cargado exitosamente desde el disco.
DatasetDict({
    train: Dataset({
        features: ['input', 'target'],
        num_rows: 624
    })
    test: Dataset({
        features: ['input', 'target'],
        num_rows: 1000
    })
})

--- Cargando dataset sintético desde: /content/drive/MyDrive/Colab Notebooks/Proyecto_DL_Traduccion/en_quc_dataset_files ---
✅ Dataset sintético Inglés-K'iche' cargado correctamente con 2 columnas.


In [59]:
for i in range(0, 20):
    print(f"> Input:     {es_quc_dataset['test'][i]['input']}")
    print(f"> Target:  {es_quc_dataset['test'][i]['target']}")
    print("-")

> Input:     Kqʼulqʼut ri apam
> Target:  #quc# Tienes indigestión
-
> Input:     Xanimaj bʼik ri kuk
> Target:  #quc# La ardilla se escapó
-
> Input:     Tajin kamuxan ri awecham
> Target:  #quc# Tu cuñado está nadando
-
> Input:     Xinriq raxtew pa taqʼaj
> Target:  #quc# En la costa me contagié de paludismo
-
> Input:     Xinpe rukʼ chech ubʼanik siʼ, pa ri juyubʼ
> Target:  #quc# Vengo con él para hacer leña en la montaña
-
> Input:     Kʼo kʼaqat chi raqan ri tataʼ
> Target:  #quc# El señor tiene picazón en los pies
-
> Input:     Rajwaxik kawetaʼmaj pa uwiʼ ri chʼajchʼojisanem
> Target:  #quc# Necesitas aprender acerca el aseo
-
> Input:     Nim ri qʼoqʼ
> Target:  #quc# El chilacayote es grande
-
> Input:     Chqʼalajisaj awibʼ chi kiwach ri tinamit
> Target:  #quc# Aclárate ante el pueblo
-
> Input:     Kakowij che ri apatan
> Target:  #quc# Te apuras a tus tareas
-
> Input:     Ri abʼaj yatzalik
> Target:  #quc# La piedra es liviana
-
> Input:     Pa juyubʼ Sibʼalaj raqan neʼ

## Evaluación Comparativa de Experimentos

### Métricas

In [ ]:
rouge = evaluate.load("rouge")
bleu = evaluate.load("bleu")
meteor = evaluate.load("meteor")
comet = evaluate.load("comet")

### Experimentos

#### 1. Zero-shot

Como NLLB tiene el idioma K’iche’, se coloca este como idioma objetivo para las traducciones directas.

In [45]:
def generate_predictions(n_examples=1000):
    predictions = []
    references = []

    for item in tqdm(english_kiche_dataset["test"].select(range(n_examples))):
      input_text = item["input"]
      reference = item["target"]

      inputs = zeroshot_tokenizer(input_text, return_tensors="pt", truncation=True)
      outputs = zeroshot_model.generate(**inputs, forced_bos_token_id=forced_bos_id)
      pred = zeroshot_tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

      predictions.append(pred)
      references.append(reference)

    return predictions, references



In [46]:
n_examples = 2
predictions, references = generate_predictions(n_examples)

100%|██████████| 2/2 [00:04<00:00,  2.46s/it]


In [48]:
# Mostrar algunas predicciones
print("\n=== Ejemplos ===\n")
for i in range(n_examples):
    print(f"> Entrada:     {english_kiche_dataset['test'][i]['input']}")
    print(f"> Referencia:  {english_kiche_dataset['test'][i]['target']}")
    print(f"> Predicción:  {predictions[i]}\n")


=== Ejemplos ===

> Entrada:     Kq'ulq'ut ri apam
> Referencia:  #quc# Tienes indigestión
> Predicción:  Kq'ulq'ut ri apam

> Entrada:     Xanimaj b'ik ri kuk and other
> Referencia:  #quc# La ardilla se escapó
> Predicción:  Xanimaj b'ik ri kuk i drugi



In [ ]:
bleu_score = bleu.compute(predictions=predictions, references=[[ref] for ref in references])

print("\n=== Métricas BLEU ===")
print(f"BLEU-1: {bleu_score['precisions'][0]*100:.2f}")
print(f"BLEU-2: {bleu_score['precisions'][1]*100:.2f}")
print(f"BLEU-3: {bleu_score['precisions'][2]*100:.2f}")
print(f"BLEU-4: {bleu_score['precisions'][3]*100:.2f}")
print(f"BLEU total: {bleu_score['bleu']*100:.2f}")

rouge_score = rouge.compute(predictions=predictions, references=references)

print("\n=== Métricas ROUGE ===")
print(f"ROUGE-1:   {rouge_score['rouge1']*100:.2f}")
print(f"ROUGE-2:   {rouge_score['rouge2']*100:.2f}")
print(f"ROUGE-L:   {rouge_score['rougeL']*100:.2f}")
print(f"ROUGE-Lsum:{rouge_score['rougeLsum']*100:.2f}")


=== Métricas BLEU ===
BLEU-1: 1.03
BLEU-2: 0.07
BLEU-3: 0.00
BLEU-4: 0.00
BLEU total: 0.00

=== Métricas ROUGE ===
ROUGE-1:   0.74
ROUGE-2:   0.08
ROUGE-L:   0.72
ROUGE-Lsum:0.73


In [ ]:
with open(f"{DRIVE_PATH}/results_es_quc.tsv", "w", encoding="utf-8", newline="") as f:
    writer = csv.writer(f, delimiter="\t")
    writer.writerow(["input", "reference", "prediction"])

    for example, pred, ref in zip(es_quc_dataset["test"], predictions, references):
        writer.writerow([example["input"], ref, pred])

print("✅ Resultados guardados en 'results_es_quc.tsv'")

✅ Resultados guardados en 'results_es_quc.tsv'


#### 2. Finetuning ES-QUC y Pipeline

In [ ]:
predictions_intermediate = []
references_intermediate = []

print("\n🔹 Traduciendo: inglés → español → K’iche’ (vía intermediaria)\n")

for item in tqdm(es_quc_dataset["test"]):
    input_text = item["input"]
    reference = item["target"]

    # Paso 1: inglés → español (modelo base)
    inter = translate_text(zeroshot_model, zeroshot_tokenizer, input_text, "eng_Latn", "spa_Latn")

    # Paso 2: español → K’iche’ (modelo fine-tuneado)
    pred = translate_text(finetune_model, finetune_tokenizer, inter, "spa_Latn", "quc_Latn")

    predictions_intermediate.append(pred)
    references_intermediate.append(reference)


🔹 Traduciendo: inglés → español → K’iche’ (vía intermediaria)



  0%|          | 0/1000 [00:02<?, ?it/s]


NameError: name 'finetune_model' is not defined

In [ ]:
print("\n=== Ejemplos Fine-Tuned ===\n")
for i in range(5):
    print(f"> Entrada:     {es_quc_dataset['test'][i]['input']}")
    print(f"> Referencia:  {es_quc_dataset['test'][i]['target']}")
    print(f"> Predicción:  {predictions_intermediate[i]}\n")

In [ ]:
bleu_score_ft = bleu.compute(predictions=predictions_intermediate, references=[[r] for r in references_intermediate])
rouge_score_ft = rouge.compute(predictions=predictions_intermediate, references=references_intermediate)

print("\n=== Métricas Fine-Tuned ===")
print(f"BLEU: {bleu_score_ft['bleu']:.2f}")
print(f"ROUGE-L: {rouge_score_ft['rougeL']:.2f}")

In [ ]:
# prompt: Save the relevant results, predictions, etc

# Assuming predictions_intermediate and references_intermediate are already computed from the previous cell

output_filename_ft = f"{DRIVE_PATH}/results_finetuned_es_quc_pipeline.tsv"

with open(output_filename_ft, "w", encoding="utf-8", newline="") as f:
    writer = csv.writer(f, delimiter="\t")
    # Write header
    writer.writerow(["input_english", "intermediate_spanish", "reference_kiche", "prediction_kiche"])

    # Write data
    # Need to regenerate the intermediate Spanish translations to save them
    # or store them during the generation process
    intermediate_translations = []
    print("\n🔄 Regenerating intermediate Spanish translations for saving...")
    for item in tqdm(es_quc_dataset["test"]):
         input_text = item["input"]
         inter = translate_text(zeroshot_model, zeroshot_tokenizer, input_text, "eng_Latn", "spa_Latn")
         intermediate_translations.append(inter)


    for i in range(len(es_quc_dataset["test"])):
         writer.writerow([
              es_quc_dataset["test"][i]["input"], # Original English input from dataset
              intermediate_translations[i], # Intermediate Spanish translation
              references_intermediate[i], # Original K'iche' reference from dataset
              predictions_intermediate[i] # Final K'iche' prediction from the fine-tuned model
         ])

print(f"✅ Resultados del pipeline (inglés -> español -> K'iche') guardados en '{output_filename_ft}'")


#### 3. Finetuning ES-QUC y Traducción Directa

In [ ]:
predictions_direct = []
references_direct = []

print("\n🔹 Traduciendo: inglés → K’iche’ (directo)\n")

for item in tqdm(es_quc_dataset["test"]):
    input_text = item["input"]
    reference = item["target"]
    pred = translate_text(finetune_model, finetune_tokenizer, input_text, "eng_Latn", "quc_Latn")

    predictions_direct.append(pred)
    references_direct.append(reference)

In [ ]:
print("\n=== Ejemplos Fine-Tuned ===\n")
for i in range(5):
    print(f"> Entrada:     {es_quc_dataset['test'][i]['input']}")
    print(f"> Referencia:  {es_quc_dataset['test'][i]['target']}")
    print(f"> Predicción:  {predictions_direct[i]}\n")

In [ ]:
bleu_score_ft = bleu.compute(predictions=predictions_direct, references=[[r] for r in references_direct])
rouge_score_ft = rouge.compute(predictions=predictions_direct, references=references_direct)

print("\n=== Métricas Fine-Tuned ===")
print(f"BLEU: {bleu_score_ft['bleu']:.2f}")
print(f"ROUGE-L: {rouge_score_ft['rougeL']:.2f}")

In [ ]:
# prompt: Save the relevant results

# Assuming predictions_direct and references_direct are already computed from the previous cell

output_filename_direct = f"{DRIVE_PATH}/results_finetuned_es_quc_direct.tsv"

with open(output_filename_direct, "w", encoding="utf-8", newline="") as f:
    writer = csv.writer(f, delimiter="\t")
    # Write header
    writer.writerow(["input_english", "reference_kiche", "prediction_kiche"])

    # Write data
    for i in range(len(es_quc_dataset["test"])):
         writer.writerow([
              es_quc_dataset["test"][i]["input"], # Original English input from dataset
              references_direct[i], # Original K'iche' reference from dataset
              predictions_direct[i] # Final K'iche' prediction from the fine-tuned model
         ])

print(f"✅ Resultados de traducción directa (inglés -> K'iche') guardados en '{output_filename_direct}'")


#### 4. Finetuning EN-QUC

In [ ]:
predictions_en_qum = []
references_en_qum = []

print("\n🔹 Traduciendo: inglés → K’iche’ (dataset sintético directo)\n")

for item in tqdm(en_qum_dataset["test"]):
    input_text = item["input"]
    reference = item["target"]
    pred = translate_text(en_qum_model, en_qum_tokenizer, input_text, "eng_Latn", "quc_Latn")

    predictions_en_qum.append(pred)
    references_en_qum.append(reference)

In [ ]:


print("\n=== Ejemplos Fine-Tuned ===\n")
for i in range(5):
    print(f"> Entrada:     {en_qum_dataset['test'][i]['input']}")
    print(f"> Referencia:  {en_qum_dataset['test'][i]['target']}")
    print(f"> Predicción:  {predictions_en_qum[i]}\n")



In [ ]:


bleu_score = bleu.compute(predictions=predictions_en_qum, references=[[ref] for ref in references_en_qum])

print("\n=== Métricas BLEU ===")
print(f"BLEU-1: {bleu_score['precisions'][0]*100:.2f}")
print(f"BLEU-2: {bleu_score['precisions'][1]*100:.2f}")
print(f"BLEU-3: {bleu_score['precisions'][2]*100:.2f}")
print(f"BLEU-4: {bleu_score['precisions'][3]*100:.2f}")
print(f"BLEU total: {bleu_score['bleu']*100:.2f}")

rouge_score = rouge.compute(predictions=predictions_en_qum, references=references_en_qum)

print("\n=== Métricas ROUGE ===")
print(f"ROUGE-1:   {rouge_score['rouge1']*100:.2f}")
print(f"ROUGE-2:   {rouge_score['rouge2']*100:.2f}")
print(f"ROUGE-L:   {rouge_score['rougeL']*100:.2f}")
print(f"ROUGE-Lsum:{rouge_score['rougeLsum']*100:.2f}")

# meteor_score = meteor.compute(predictions=predictions_en_qum, references=references_en_qum)

# print("\n=== Otras Métricas ===")
# print(f"METEOR:     {meteor_score['meteor']*100:.2f}")



In [ ]:


filename = "results_en_qum_directo.tsv"
with open(filename, "w", encoding="utf-8", newline="") as f:
    writer = csv.writer(f, delimiter="\t")
    writer.writerow(["input", "reference", "prediction"])

    for example, pred, ref in zip(es_qum_dataset["test"], predictions_en_qum, references_en_qum):
        writer.writerow([example["input"], ref, pred])

print(f"✅ Resultados guardados en '{filename}'")

